In [1]:
# Imports
import json
from collections import OrderedDict

Error: Jupyter cannot be started. Error attempting to locate jupyter: Data Science libraries jupyter and notebook are not installed in interpreter Python 3.9.0 64-bit.

## Definitions

### Transitions Definition

In [36]:
# Json input
input_str= """
{
    "T1":[
        ["P1", "P2"],
        [
            [ "I1 & I2 & !I4" ],
            [ ["I4"], ["I3"] ]
        ],
        ["P3"],
        "T1 comment"
    ],
    "T2":[
        ["P3"],
        [
            [ ["I4"], ["I1", "I2"] ],
            [ "I4 & !I3" ]
        ],
        ["P1", "P2"],
        "T2 comment"
    ]
}
"""

In [37]:
# Convert Json string to python dictionary
in_dict= json.loads(input_str, object_pairs_hook=OrderedDict)

In [38]:
# Expected NuSMV
expected= "T1:= (P1 & P2) & ( (I1 & I2 & !I4) | (I4 & !I3) ) & (!P3);\n" + \
"T2:= (P3) & ( (I4 & !I1 & !I2) | (I4 & !I3) ) & (!P1 & !P2);\n"  

In [44]:
out_str= ""

In [45]:
value= in_dict["T1"]

In [46]:
value[1][0]

['I1 & I2 & !I4']

In [47]:
for key in iter(in_dict):

    # Get value of current transition
    value= in_dict[key]

    # Pre-places
    pre= value[0]
    # Pre-places condition
    pre_str= "("
    for idx,item in enumerate(pre):
        # Not last element
        if idx < len(pre)-1:
            pre_str= pre_str + item + " & "
        else:
            pre_str= pre_str + item

    pre_str= pre_str + ")"

    # Inputs
    req_inputs= value[1]
    # Required inputs condition
    req_str= "( "
    for idx_out,item in enumerate(req_inputs):

        # Temp string opening brace
        temp_str= "("

        # Check if condition is specified as a raw string
        if isinstance(item[0], str):
            temp_str= temp_str + item[0]
        
        else:
            high_ins= item[0]
            low_ins= item[1]
            # temp inputs
            for idx_high,high_in in enumerate(high_ins):
                # Not last item
                if idx_high < len(high_ins)-1:
                    temp_str= temp_str + high_in + " & "
                # If last item
                else:
                    # If there are no low outputs required
                    if len(low_ins) < 1:
                        temp_str= temp_str + high_in
                    else:
                        temp_str= temp_str + high_in + " & "
            
            # Low inputs
            for idx_low,low_in in enumerate(low_ins):
                # Not last item
                if idx_low < len(low_ins)-1:
                    temp_str= temp_str + "!" + low_in + " & "
                # If last item
                else:
                    temp_str= temp_str + "!" + low_in

        # Temp string closing brace
        temp_str= temp_str + ")"   

        # If not last item, add an OR operator
        if idx_out < (len(req_inputs)-1):
            temp_str= temp_str + " | "

        # Append Temp string to req_str
        req_str= req_str + temp_str

    # req_str closin brace
    req_str= req_str + " )"

    # Post-places
    post= value[2]
    # Post-places condition
    post_str= "("
    for idx_post,item_post in enumerate(post):
        # Not last element
        if idx_post < len(post)-1:
            post_str= post_str + "!" + item_post + " & "
        else:
            post_str= post_str + "!" + item_post

    post_str= post_str + ")"

    # Append transition name to output
    out_str= out_str + key + ":= "
    # Append pre-place condition
    out_str= out_str + pre_str + " & "
    # Append inputs condition
    out_str= out_str + req_str + " & "
    # Append post-place condition
    out_str= out_str + post_str + ";\n"

In [2]:
out_str

NameError: name 'out_str' is not defined

In [49]:
expected

'T1:= (P1 & P2) & ( (I1 & I2 & !I4) | (I4 & !I3) ) & (!P3);\nT2:= (P3) & ( (I4 & !I1 & !I2) | (I4 & !I3) ) & (!P1 & !P2);\n'

In [50]:
assert(out_str == expected)

### Stability Definition

In [2]:
# Json input
input_str= """
{
    "T1":[
        ["P1", "P2"],
        [
            [ ["I1", "I2"], ["I4"] ],
            [ ["I3"], ["I4"] ]
        ],
        ["P3"]
    ],
    "T2":[
        ["P3"],
        [
            [ ["I4"], ["I1", "I2"] ],
            [ ["I4"], ["I3"] ]
        ],
        ["P1", "P2"]
    ]
}
"""

In [3]:
# Convert Json string to python dictionary
input_dict= json.loads(input_str, object_pairs_hook=OrderedDict)

In [49]:
# Expected NuSMV
expected= "stab:= !(T1 | T2);\n"

In [46]:
def stab_definition(in_dict):
    """Define stability checking condition in NuSMV

    Parameters:
    ----------
    in_dict: dict

    Returns:
    -------
    out_str: string
    """
    out_str="stab:= !("

    keys= list(in_dict)
    for idx,key in enumerate(keys):
        # If not last item
        if idx < len(keys)-1:
            # Append transition name and add OR
            out_str= out_str + key + " | "
        else:
            # Append transition name
            out_str= out_str + key

    # Add closing brace and semicolon
    out_str= out_str + ");\n"

    return out_str


In [47]:
stab_definition(input_dict)

'stab:= !(T1 | T2);\n'

In [50]:
expected

'stab:= !(T1 | T2);\n'

### Output Definition

In [33]:
# Json input
input_str= """
{
    "PS2": [
        [
            ["RP_AL1_ST_CLAMP"], ["AL1_ST_GRAB"]
        ],
        "Product gets withing graber range"
    ],
    "PS4": [
        [
            ["RP_AL1_ST_CLAMP"], ["AL1_ST_GRAB"]
        ],
        ["AL1_ST_Z_SET:=750"],
        "Move grabber down to pick up product"
    ],
    "PSE0": [
        [
            ["AL1_ST_GRAB"], ["AL1_EMIT", "RC_AL1_ST"]
        ],
        "New box arrived, stop conveyor"
    ],
    "initial": ["PS2", "PSE0"]
    
}
"""

In [95]:
# Expected output
expected= "-- SET\n" + \
    "AL1_EMIT_set:= FALSE;\n" + \
    "AL1_ST_GRAB_set:= PSE0;\n" + \
    "RC_AL1_ST_set:= FALSE;\n" + \
    "RP_AL1_ST_CLAMP_set:= PS2 | PS4;\n" + \
    "-- RESET\n" + \
    "AL1_EMIT_reset:= PSE0;\n" + \
    "AL1_ST_GRAB_reset:= PS2 | PS4;\n" + \
    "RC_AL1_ST_reset:= PSE0;\n" + \
    "RP_AL1_ST_CLAMP_reset:= FALSE;\n" + \
    "-- OUTPUT\n" + \
    "AL1_EMIT:= AL1_EMIT_set & !AL1_EMIT_reset;\n" + \
    "AL1_ST_GRAB:= AL1_ST_GRAB_set & !AL1_ST_GRAB_reset;\n" + \
    "RC_AL1_ST:= RC_AL1_ST_set & !RC_AL1_ST_reset;\n" + \
    "RP_AL1_ST_CLAMP:= RP_AL1_ST_CLAMP_set & !RP_AL1_ST_CLAMP_reset;\n" + \
    "-- Non-Boolean Outputs\n" + \
    "AL1_ST_Z_SET:= case\n" + \
    "   PS4: 750;\n" + \
    "   TRUE: 0;\n" + \
    "esac;\n"

In [66]:
# Convert Json string to python dictionary
in_dict= json.loads(input_str, object_pairs_hook=OrderedDict)


In [67]:
in_dict

OrderedDict([('PS2',
              [[['RP_AL1_ST_CLAMP'], ['AL1_ST_GRAB']],
               'Product gets withing graber range']),
             ('PS4',
              [[['RP_AL1_ST_CLAMP'], ['AL1_ST_GRAB']],
               ['AL1_ST_Z_SET:=750'],
               'Move grabber down to pick up product']),
             ('PSE0',
              [[['AL1_ST_GRAB'], ['AL1_EMIT', 'RC_AL1_ST']],
               'New box arrived, stop conveyor']),
             ('initial', ['PS2', 'PSE0'])])

In [68]:
in_dict["PS4"][0]

[['RP_AL1_ST_CLAMP'], ['AL1_ST_GRAB']]

In [69]:
# Get set of boolean outputs
output_set= set()

for key in list(in_dict):
    if not key == "initial":
        for output in in_dict[key][0][0]:
            output_set.add(output)
        for output in in_dict[key][0][1]:
            output_set.add(output)


In [70]:
# Sort alphabetically
output_set= sorted(output_set)

In [71]:
print(output_set)

['AL1_EMIT', 'AL1_ST_GRAB', 'RC_AL1_ST', 'RP_AL1_ST_CLAMP']


In [72]:
# Collate list of places for which each output is set/reset
output_dict= OrderedDict()

for output in output_set:
    # Initialize list of places for which output is set/reset
    output_dict[output]= [[],[]]

    for key in list(in_dict):
        # Add places for which output is set
        if output in in_dict[key][0][0]:
            output_dict[output][0].append(key)

        # Add places for which output is reset
        if output in in_dict[key][0][1]:
            output_dict[output][1].append(key)


In [73]:
print(output_dict)

OrderedDict([('AL1_EMIT', [[], ['PSE0']]), ('AL1_ST_GRAB', [['PSE0'], ['PS2', 'PS4']]), ('RC_AL1_ST', [[], ['PSE0']]), ('RP_AL1_ST_CLAMP', [['PS2', 'PS4'], []])])


In [74]:
# Generate output string
set_output_str= "-- SET\n"
reset_output_str= "-- RESET\n"
output_str= "-- OUTPUT\n"

# Loop through output keys
for key in list(output_dict):
    output= output_dict[key]

    # Get set places
    set_output_str= set_output_str +  key + "_set:= "
    # If the output is set at at least one place
    if len(output[0]) > 0:
        for idx_set,place_set in enumerate(output[0]):
            # Not last item
            if idx_set < len(output[0])-1:
                set_output_str= set_output_str + output[0][idx_set] + " | "
            # Last Item
            else:
                set_output_str= set_output_str + output[0][idx_set] + ";\n"
    # If output is not set at any place
    else:
        set_output_str= set_output_str + "FALSE;\n"

    # Get reset places
    reset_output_str= reset_output_str +  key + "_reset:= "
    # If output is reset at at least 1 place
    if len(output[1]) > 0:
        for idx_reset,place_reset in enumerate(output[1]):
            # Not last item
            if idx_reset < len(output[1])-1:
                reset_output_str= reset_output_str + output[1][idx_reset] + " | "
            # Last Item
            else:
                reset_output_str= reset_output_str + output[1][idx_reset] + ";\n"

    # If output is not reset at any place
    else:
        reset_output_str= reset_output_str + "FALSE;\n"


    # Compose output string
    output_str= output_str + key + ":= " + key + "_set & !" + key + "_reset;\n" 


In [77]:
out_str= set_output_str + reset_output_str + output_str

### Non-boolean outputs

In [81]:
in_dict

OrderedDict([('PS2',
              [[['RP_AL1_ST_CLAMP'], ['AL1_ST_GRAB']],
               'Product gets withing graber range']),
             ('PS4',
              [[['RP_AL1_ST_CLAMP'], ['AL1_ST_GRAB']],
               ['AL1_ST_Z_SET:=750'],
               'Move grabber down to pick up product']),
             ('PSE0',
              [[['AL1_ST_GRAB'], ['AL1_EMIT', 'RC_AL1_ST']],
               'New box arrived, stop conveyor']),
             ('initial', ['PS2', 'PSE0'])])

In [85]:
not in_dict["PS4"][1]

False

In [92]:
# Non-boolean outputs
# Get dictionary of outputs, the places that set them, and their assigned values
output_dict= dict()

for key in list(in_dict):
    if not key == "initial":
        # Check if exists
        if in_dict[key][1] and isinstance(in_dict[key][1], list):
            # Get string
            val= in_dict[key][1][0]
            # Split string on :=
            output_name, aa, output_val= val.partition(":=") 
            output_dict[output_name]= [key, output_val]


In [93]:
output_dict

{'AL1_ST_Z_SET': ['PS4', '750']}

In [99]:
nb_output_str= "-- Non-Boolean Outputs\n"

In [100]:
# Compose string to set outputs
for key in list(output_dict):
    nb_output_str= nb_output_str + key + ":= case\n" + \
        "   " + output_dict[key][0] + ": " + output_dict[key][1] + ";\n" + \
        "   TRUE: 0;\n" + \
        "esac;\n"

In [101]:
nb_output_str

'-- Non-Boolean Outputs\nAL1_ST_Z_SET:= case\n   PS4: 750;\n   TRUE: 0;\nesac;\n'

In [102]:
out_str= out_str + nb_output_str

In [103]:
out_str

'-- SET\nAL1_EMIT_set:= FALSE;\nAL1_ST_GRAB_set:= PSE0;\nRC_AL1_ST_set:= FALSE;\nRP_AL1_ST_CLAMP_set:= PS2 | PS4;\n-- RESET\nAL1_EMIT_reset:= PSE0;\nAL1_ST_GRAB_reset:= PS2 | PS4;\nRC_AL1_ST_reset:= PSE0;\nRP_AL1_ST_CLAMP_reset:= FALSE;\n-- OUTPUT\nAL1_EMIT:= AL1_EMIT_set & !AL1_EMIT_reset;\nAL1_ST_GRAB:= AL1_ST_GRAB_set & !AL1_ST_GRAB_reset;\nRC_AL1_ST:= RC_AL1_ST_set & !RC_AL1_ST_reset;\nRP_AL1_ST_CLAMP:= RP_AL1_ST_CLAMP_set & !RP_AL1_ST_CLAMP_reset;\n-- Non-Boolean Outputs\nAL1_ST_Z_SET:= case\n   PS4: 750;\n   TRUE: 0;\nesac;\n'

In [104]:
expected

'-- SET\nAL1_EMIT_set:= FALSE;\nAL1_ST_GRAB_set:= PSE0;\nRC_AL1_ST_set:= FALSE;\nRP_AL1_ST_CLAMP_set:= PS2 | PS4;\n-- RESET\nAL1_EMIT_reset:= PSE0;\nAL1_ST_GRAB_reset:= PS2 | PS4;\nRC_AL1_ST_reset:= PSE0;\nRP_AL1_ST_CLAMP_reset:= FALSE;\n-- OUTPUT\nAL1_EMIT:= AL1_EMIT_set & !AL1_EMIT_reset;\nAL1_ST_GRAB:= AL1_ST_GRAB_set & !AL1_ST_GRAB_reset;\nRC_AL1_ST:= RC_AL1_ST_set & !RC_AL1_ST_reset;\nRP_AL1_ST_CLAMP:= RP_AL1_ST_CLAMP_set & !RP_AL1_ST_CLAMP_reset;\n-- Non-Boolean Outputs\nAL1_ST_Z_SET:= case\n   PS4: 750;\n   TRUE: 0;\nesac;\n'

In [105]:
assert (out_str == expected)